## Imports

In [1]:
from __future__ import annotations
from typing import Dict, List, Union

import logging
import os
import random
import sys
from collections import deque
from operator import itemgetter

import gym_donkeycar
import gymnasium as gym
import imageio
import ipywidgets as widgets
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from IPython.display import display
from ipywidgets import HBox, VBox
from matplotlib import pyplot as plt
from PIL import Image
from ruamel.yaml import YAML
from scipy.ndimage import gaussian_filter1d
from scipy.stats import norm
from tensorboard import notebook
from tensorboard.backend.event_processing.event_accumulator import \
    EventAccumulator
from torch import distributions as dist
from torch.distributions import Categorical, Normal
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from tqdm import tqdm

import gymnasium as gym

# suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="gymnasium.spaces.box") # module="gymnasium"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["IMAGEIO_IGNORE_WARNINGS"] = "True"

import stable_baselines3 as sb3
from gym_donkeycar.envs.donkey_env import DonkeyEnv
from gymnasium import spaces
from gymnasium.spaces import Box
from stable_baselines3 import A2C, PPO, SAC
from stable_baselines3.common import env_checker
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv

import src
from src.actor_critic_discrete import DiscreteActorCritic
from src.actor_critic_dreamer import ActorCriticDreamer
from src.actor_critic import ContinuousActorCritic
from src.blocks import CategoricalStraightThrough, ConvBlock
from src.categorical_vae import CategoricalVAE
from src.imagination_env import make_imagination_env
from src.mlp import MLP
from src.preprocessing import transform
from src.replay_buffer import ReplayBuffer
from src.rssm import RSSM
from src.utils import (load_config, make_env, save_image_and_reconstruction,
                       to_np, symlog, symexp, twohot_encode, ExponentialMovingAvg,
                       ActionExponentialMovingAvg, MetricsTracker)
from src.vae import VAE

torch.cuda.empty_cache()
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Load the config
config = load_config()
for key in config:
    locals()[key] = config[key]

## Create the environment

In [2]:
env = gym.make("Pendulum-v1")
env = gym.wrappers.RecordEpisodeStatistics(env, deque_size=config["n_updates"])
env = gym.wrappers.TimeLimit(env, max_episode_steps=config["max_episode_steps"])
env = gym.wrappers.AutoResetWrapper(env)
env = gym.experimental.wrappers.RescaleActionV0(env, min_action=config["action_space_low"], max_action=config["action_space_high"])

In [3]:
# agent = ContinuousActorCritic()
# agent = ActorCriticDreamer()
agent = DiscreteActorCritic()

# agent.load_weights("weights/ContinuousActorCritic_0")

# vae = VAE()
# vae.optim = optim.Adam(vae.parameters(), lr=1e-4, weight_decay=1e-6)

Initializing critic.
Adding zero weight init to the output layer.

Initializing actor.


In [ ]:
# New training loop with batches for the distributional critic

tracker = MetricsTracker(
    training_metrics=["critic_loss", "actor_loss"],
    episode_metrics=["rewards", "log_probs", "value_preds", "critic_dists", "entropies", "masks"],
)

for sample_phase in tqdm(range(n_updates)):
    
    if sample_phase == 0:
        obs, info = env.reset(seed=42)
        obs = torch.tensor(obs).to(device) ### Pendulum
        # obs = transform(obs)
        # if len(obs.shape) == 4:
        #     obs = obs[0]
        # z, reconstruction, mu, logvar = vae(obs)

    for step in range(n_steps_per_update):
            
        value_pred, critic_dist = agent.apply_critic(obs) # obs for Pendulum | z for VAE

        # Get an action and take an environment step
        action, log_prob, actor_entropy = agent.get_action(obs) ### obs for Pendulum | z for VAE
        obs, reward, terminated, truncated, info = env.step(to_np(action))
        
        # Transform the next obs
        obs = torch.Tensor(obs).to(device) ### Pendulum
        # obs = transform(obs)
        # if len(obs.shape) == 4:
        #     obs = obs[0]
        # z, reconstruction, mu, logvar = vae(obs)
        
        # every step:
        tracker.add(
            episode_metrics={
                "rewards": reward,
                "log_probs": log_prob,
                "value_preds": value_pred,
                "critic_dists": critic_dist,
                "entropies": actor_entropy,
                "masks": torch.tensor(0.0 if terminated else 1.0),
            }
        )
    
    # every sample phase:
    episode_batches = tracker.get_episode_batches() # episode_batches is a dict
    last_value_pred, _ = agent.apply_critic(obs) # last value prediction for GAE

    # Update the agent's parameters
    critic_loss, actor_loss = agent.get_loss(episode_batches, last_value_pred)
    agent.update_parameters(critic_loss, actor_loss)
    
    # Log the episode metrics
    if sample_phase % config["log_interval"] == 0:
        tracker.add(
            training_metrics={
                "critic_loss": critic_loss,
                "actor_loss": actor_loss,
            }
        )

        # Episode return
        if len(env.return_queue):
            tracker.writer.add_scalar("episode_return", np.array(env.return_queue)[-1], global_step=len(env.return_queue))

        # Actor and critic losses
        tracker.log_to_tensorboard(step=sample_phase)
        